## COVID-19
### Exploring Corona Virus Publications with WeLearn.

Hey there! This is a notebook I used for doing curated page for WeLearn.

If you haven't, check out the [Map on WeLearn][exp-page].

This is an open source analysis, and available on GitHub.

All the data used for this analysis is available via [WeLearn API][api-url].


[api-url]: https://welearn.cri-paris.org/.meta/docs
[exp-page]: https://welearn.cri-paris.org/experiments/covid19

In [1]:
%config InlineBackend.figure_formats = ['png']

import requests
import pandas as pd
import numpy as np
import scipy as sci
import scipy.spatial
import scipy.linalg
import networkx as nx
import networkx.readwrite.json_graph

from urllib.parse import urlparse
from pprint import pprint

In [2]:
COVID19_UUID = '8705149e0d3a449e9a70747df29ccea4'
API_BASE_URL = f'https://welearn.cri-paris.org/api/resources/user/{COVID19_UUID}?limit=500'

covid_resources = requests.get(API_BASE_URL).json()
resources = covid_resources['results']

print(f'n(Resources) = {len(resources)}\n')

pprint(resources[0])

n(Resources) = 203

{'concepts': [{'cuid': 'Q28946449',
               'title_en': 'Transcriptomics technologies',
               'title_fr': None,
               'wikidata_id': 'Q28946449'},
              {'cuid': 'Q7318015',
               'title_en': 'Revenue assurance',
               'title_fr': None,
               'wikidata_id': 'Q7318015'},
              {'cuid': 'Q858810',
               'title_en': 'Big data',
               'title_fr': 'Big data',
               'wikidata_id': 'Q858810'},
              {'cuid': 'Q5282133',
               'title_en': 'Disease surveillance in China',
               'title_fr': None,
               'wikidata_id': 'Q5282133'},
              {'cuid': 'Q891528',
               'title_en': 'Social media measurement',
               'title_fr': None,
               'wikidata_id': 'Q891528'}],
 'created': '2020-03-04T16:28:22.901685+00:00',
 'lang': 'en',
 'readability_score': 42.0,
 'resource_id': '91c1b55c623d46e08b05e62d7c77314e',
 'title': 'Novel

### Using NetworkX for Graph Statistics

The API gets us the resources in this format:

```json
{
  "results": [
    {
      "resource_id": "string",
      "title": "string",
      "url": "string",
      "lang": "string",
      "readability_score": 0,
      "concepts": [
        {
          "cuid": "string",
          "wikidata_id": "string",
          "title_en": "string",
          "title_fr": "string"
        }
      ]
    }
  ],
  "pagination": {
    "count": 0,
    "skip": 0,
    "limit": 0,
    "next": 0
  }
}
```

We can quite easily transform this to a `node-link` format object to get a `NetworkX` Graph.

The `node-link` format looks like this:
```json
{ nodes: [{id, ...props}], links: [{source, target}] }
```

In [3]:
def resource_graph(data):
  def links_gen():
    for resource in data:
      for concept in resource['concepts']:
        yield {
          'source': resource['resource_id'],
          'target': concept['wikidata_id'],
        }

  def nodes_gen():
    '''Here we're sequentially emitting the resource and concept nodes.
       It should be noted that this is a flat list so we add the "group"
       properties as well.
       Group 1: Resource, Group 2: Concept.

       For concept nodes, we'd like to avoid duplicating data. We'll use
       a list to keep track of the concepts already emitted.
    '''
    _cemit = []
    for resource in data:
      yield {
        'id': resource['resource_id'],
        'group': 1,
        'url': resource['url'],
        'title': resource['title'],
      }
      for concept in resource['concepts']:
        if concept['wikidata_id'] in _cemit:
          continue

        _cemit.append(concept['wikidata_id'])
        yield {
          'id': concept['wikidata_id'],
          'group': 2,
          'title_en': concept['title_en'],
          'title_fr': concept['title_fr'],
        }

  node_links = {
    'nodes': list(nodes_gen()),
    'links': list(links_gen()),
  }
  graph = networkx.json_graph.node_link_graph(node_links)
  
  return (graph, node_links)


G, node_link_graph = resource_graph(resources)

pprint(node_link_graph)

{'links': [{'source': '91c1b55c623d46e08b05e62d7c77314e',
            'target': 'Q28946449'},
           {'source': '91c1b55c623d46e08b05e62d7c77314e', 'target': 'Q7318015'},
           {'source': '91c1b55c623d46e08b05e62d7c77314e', 'target': 'Q858810'},
           {'source': '91c1b55c623d46e08b05e62d7c77314e', 'target': 'Q5282133'},
           {'source': '91c1b55c623d46e08b05e62d7c77314e', 'target': 'Q891528'},
           {'source': 'd36ab020de1c41b0bcca354702ab5269', 'target': 'Q5382721'},
           {'source': 'd36ab020de1c41b0bcca354702ab5269', 'target': 'Q494348'},
           {'source': 'd36ab020de1c41b0bcca354702ab5269',
            'target': 'Q39087737'},
           {'source': 'd36ab020de1c41b0bcca354702ab5269', 'target': 'Q1741825'},
           {'source': 'd36ab020de1c41b0bcca354702ab5269', 'target': 'Q5938794'},
           {'source': 'd36ab020de1c41b0bcca354702ab5269', 'target': 'Q427883'},
           {'source': 'd36ab020de1c41b0bcca354702ab5269',
            'target': 'Q30688

           {'group': 2,
            'id': 'Q18208050',
            'title_en': 'Infection in childcare',
            'title_fr': None},
           {'group': 2,
            'id': 'Q6030308',
            'title_en': 'Influenza prevention',
            'title_fr': None},
           {'group': 1,
            'id': '32eab35c2329460fa03f5f06925a9a34',
            'title': 'Prevention, Treatment of Coronavirus Disease 2019 '
                     '(COVID-19) | CDC',
            'url': 'https://www.cdc.gov/coronavirus/2019-ncov/about/prevention-treatment.html'},
           {'group': 2,
            'id': 'Q520181',
            'title_en': 'Hand sanitizer',
            'title_fr': 'Solution hydro-alcoolique'},
           {'group': 2,
            'id': 'Q552461',
            'title_en': 'Hand washing',
            'title_fr': 'Hygiène des mains'},
           {'group': 2,
            'id': 'Q18344466',
            'title_en': 'Prevention of viral hemorrhagic fever',
            'title_fr': None},
  

            'title_en': 'H II region',
            'title_fr': 'Région HII'},
           {'group': 2,
            'id': 'Q7932942',
            'title_en': 'Vinyl cation',
            'title_fr': None},
           {'group': 1,
            'id': '4484d57118ec4fbcad0e4482e8408db6',
            'title': 'מחשש למחסור: משרד הבריאות הורה לקופות ובתי חולים להצטייד '
                     'בתרופות לחצי שנה - בריאות - TheMarker',
            'url': 'https://www.themarker.com/news/health/1.8595771'},
           {'group': 1,
            'id': '42f13d328b9644ad8b7cefc6bd813b31',
            'title': 'Coronavirus could lead to drug shortages in US - ABC '
                     'News',
            'url': 'https://abcnews.go.com/Health/coronavirus-lead-drug-shortages-us/story?id=69243037'},
           {'group': 1,
            'id': 'b949a936f1a646d0a739a5a1e37c13ab',
            'title': '665AFA1E710B4B3293138D14BE8D1213.pdf',
            'url': 'https://www.acc.org/~/media/665AFA1E710B4B3293138D14BE8D

            'title': 'Event 201 videos including discussions and a highlights '
                     'reel',
            'url': 'http://www.centerforhealthsecurity.org/event201/videos.html'},
           {'group': 1,
            'id': '64bb80514d2740a49a8af72300091e7d',
            'title': 'Bill & Melinda Gates Foundation Commits $10 Million to '
                     'Global Response to 2019-nCOV - Bill & Melinda Gates '
                     'Foundation',
            'url': 'https://www.gatesfoundation.org/Media-Center/Press-Releases/2020/01/Gates-Foundation-Commits-10-Million-to-Global-Response-to-2019-nCOV'},
           {'group': 2,
            'id': 'Q25037227',
            'title_en': 'Timeline of global health',
            'title_fr': None},
           {'group': 2,
            'id': 'Q929451',
            'title_en': 'Neglected tropical diseases',
            'title_fr': 'Maladies tropicales négligées'},
           {'group': 1,
            'id': '8c06cf4475cd4e82b278e4b824660dc0'

Nice! We now have the node-link format object. Lets get some stats out of this:

In [4]:
_nedge, _nnode = G.number_of_edges(), G.number_of_nodes()

print(f'n(edges) = {_nedge}')
print(f'n(nodes) = {_nnode}')


n(edges) = 679
n(nodes) = 479


**What are websites that are referred most often?**

In [9]:
# Use urllib to parse the urls and get the domain names in a set.
# with this, make a dataframe with <domain, freq> ordered pairs.

domains = [urlparse(r['url']).netloc for r in resources]
freq_list = [(x, domains.count(x)) for x in set(domains)]
domain_freq = (pd
               .DataFrame(freq_list, columns=['domain', 'freq'])
               .sort_values(by='freq', ascending=False))

domain_freq[domain_freq.freq > 2]

,domain,freq
32,www.youtube.com,11
62,twitter.com,9
56,www.bbc.com,7
78,www.bloomberg.com,6
102,www.nytimes.com,5
110,edition.cnn.com,5
20,www.biorxiv.org,4
39,www.npr.org,4
108,www.theguardian.com,4
86,www.gatesnotes.com,3
